# Face Recognition with a Support Vector Machine

Qin Zhao PhD & Bas Michielsen MSc

Face recognition is the ability for the computer to identify a person based on an image of their face. This is a classification problem, where each possible person is a class, and the provided image should lead to 1 specific class with a as high as feasible certainty. In order to train a classification model with this, we need as many as possible images of the same person's face. In this exercise we use a dataset that is provided by sklearn called the 'Olivetti Faces'. Mote information can be found at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html

The dataset has 400 images for 40 different persons, namely exactly 10 images per person. The dataset is also labelled, so for every image we know the correct class. Now we can use this dataset to train a model.

We start by importing scikit-learn, numpy, matplotlib and seaborn the Python libraries we will be using for this analysis. 

First, we show the versions of these libraries (that is always wise to do in case you have to report problems running the notebooks!) and use the inline plotting mode statement.

In [62]:
import sklearn as sk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

print('scikit-learn version:', sk.__version__)
print('numpy version:', np.__version__)
print('matplotlib version:', matplotlib.__version__)
print('seaborn version:', sns.__version__)

scikit-learn version: 1.0.2
numpy version: 1.21.5
matplotlib version: 3.5.2
seaborn version: 0.11.2


Then let us load the data.

In [63]:
from sklearn.datasets import fetch_olivetti_faces
faces = fetch_olivetti_faces(data_home= "./data")

To get an impression of the data, here we show the first photo of each of the 40 unique people in the dataset.

In [64]:
fig, subplots = plt.subplots(nrows=4, ncols=10, figsize=(18, 9))
subplots = subplots.flatten()
    
for id in np.unique(faces.target):
    index = id *10 # Because there are 10 images per person.
    subplots[id].imshow(faces.images[index], cmap="gray")
    subplots[id].set_xticks([])
    subplots[id].set_yticks([])
    subplots[id].set_title("id: {}".format(id))
plt

<module 'matplotlib.pyplot' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py'>

## Preprocessing
Given that we are using images, there is no such thing as feature selection because you cannot select some pixels to be better indicators than other pixels. We therefore have little to do in terms of preprocessing other than splitting the dataset into a trainset and testset.
We select a stratisfied test size of .3, which means that from every person 30% of the images will be used as testset. Since every person has 10 images, 7 will be selected for training and 3 will be selected for testing. 

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(faces.data, faces.target, test_size=.3, random_state=0, stratify=faces.target)

## Modelling
In this step we are going to use the trainset only to fit the model. In this case a Support Vector Machine for classification.

In [66]:
from sklearn.svm import SVC # SVC is for classification, an SVR for regression exists as well.
model = SVC()
model.fit(X_train, y_train)

SVC()

After that we can let the model do predictions on the testset and see the accuracy of those predictions.

In [67]:
pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
acc = accuracy_score(pred, y_test)
print(acc)

0.8833333333333333


An accuracy of a little more than 88% is not too bad. In fact this is quite good. ðŸ˜Š

## Evaluation
Now let us see if we can shed some light on the results. The first thing we can do here is print a classification report. This shows for every one of the classes how well the model performed. There are 2 metrics to consider, namely Precision and Recall. The F1-score is a quality score based on precision and recall together.

Precision: When the model predicts class X, how often was it correct?

Recall: When the correct class is X, how often did the model predict it so?

Note that both Precision and Recall are fractions, so 1.00 is 100% of the times, and 0.00 is 0% of the times.

In [68]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.50      0.33      0.40         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67         3
           3       0.67      0.67      0.67         3
           4       0.67      0.67      0.67         3
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         3
           7       0.67      0.67      0.67         3
           8       1.00      1.00      1.00         3
           9       1.00      0.67      0.80         3
          10       1.00      1.00      1.00         3
          11       1.00      1.00      1.00         3
          12       1.00      0.67      0.80         3
          13       1.00      1.00      1.00         3
          14       0.75      1.00      0.86         3
          15       1.00      1.00      1.00         3
          16       0.75      1.00      0.86         3
          17       1.00    

It appears that class 39 has a fairly low Precision, this means that often the model says 39 when in fact it is another person. In other words, quite a number of people get identified as person 39 incorrectly. Also it appears that the classes 0, 25 and 34 have a low Recall, this means that these people often get identified as some else.

The next thing we can do here is making a confusion matrix. This matrix shows all 40 ids (0 - 39) as rows and on every row shows the made predictions for that id in the testset. If the prediction is correct if will show in the [id, id] cell, if not, it will show in another cell and we can see the id that the model predicted instead. Since the testset is stratisfied at 3 images per person, 3 is the maximum of correct classifications, so that should be the vmax parameter.

In [69]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, pred)
plt.figure(figsize=(15, 15)) # A size of 15 because it feels good! You can try other numbers if you want.
sns.heatmap(matrix, annot=True, cbar=None, vmax=3)

<AxesSubplot:>

Of course here we see a similar result as the classification report. In the vast majority of cases the model identifies all 3 images of the same person correctly. However, the column 39 shows many 1s for other rows than 39, and the rows 0, 25 and 34 show many 1s in columns other than the correct one.

# Cross validation

In [70]:
from sklearn.model_selection import cross_val_score as cv_score
X = faces.data
y = faces.target
scores = cv_score(model, X, y)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores

Accuracy: 0.94 (+/- 0.06)


array([0.9875, 0.95  , 0.95  , 0.9375, 0.8875])

# StratifiedKFold

In [71]:
from sklearn.model_selection import StratifiedKFold as SKFold
skf = SKFold(n_splits=5, shuffle=True, random_state=1)
skf.get_n_splits(X, y)
lst_accu_stratified = []
fold_no = 1

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    model.fit(X_train_fold, y_train_fold)
    print("fold " + str(fold_no) + " accuracy: " +  str(model.score(X_test_fold, y_test_fold)))
    fold_no += 1
    lst_accu_stratified.append(model.score(X_test_fold, y_test_fold))

print('')
print('Maximum Accuracy',max(lst_accu_stratified))
print('Minimum Accuracy:',min(lst_accu_stratified))
print('Overall Accuracy:', str(round(sum(lst_accu_stratified) / len(lst_accu_stratified), 4) ))


fold 1 accuracy: 0.9375
fold 2 accuracy: 0.9
fold 3 accuracy: 0.925
fold 4 accuracy: 0.95
fold 5 accuracy: 0.95

Maximum Accuracy 0.95
Minimum Accuracy: 0.9
Overall Accuracy: 0.9325


# Glasses

In [72]:
# de index van de foto's met bril (begin, einde)
glasses = [
    (10, 19), (30, 32), (37, 38), (50, 59), (63, 64),
    (69, 69), (120, 121), (124, 129), (130, 139), (160, 161),
    (164, 169), (180, 182), (185, 185), (189, 189), (190, 192),
    (194, 194), (196, 199), (260, 269), (270, 279), (300, 309),
    (330, 339), (358, 359), (360, 369)
]


# geeft elke entry in faces.target een waarde van 0
y = np.zeros(faces.target.shape[0])

# geeft alle indexes met brillen een waarde van 1
for(start,end) in glasses:
    y[start:end + 1] = 1

# splitst de dataset in train en test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(faces.data, y, test_size=.3, random_state=0, stratify=faces.target)

In [73]:
# stopt de data in een SVC model
model.fit(X_train, y_train)

# classificeerd op test set
pred = model.predict(X_test)

#geeft een score(0.0 = 0% - 1.0 = 100%) over hoe accuraat de uitkomst van het model is op basis van de test set
from sklearn.metrics import accuracy_score
acc = accuracy_score(pred, y_test)
print(acc)

0.9833333333333333


In [74]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99        86
         1.0       1.00      0.94      0.97        34

    accuracy                           0.98       120
   macro avg       0.99      0.97      0.98       120
weighted avg       0.98      0.98      0.98       120



In [77]:
from sklearn.model_selection import cross_val_score as cv_score
X = faces.data
scores = cv_score(model, X, y)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores

Accuracy: 0.84 (+/- 0.14)


array([0.8875, 0.75  , 0.775 , 0.9375, 0.8375])

In [76]:
from sklearn.model_selection import StratifiedKFold as SKFold
skf = SKFold(n_splits=5, shuffle=True, random_state=1)
skf.get_n_splits(X, y)
lst_accu_stratified = []
fold_no = 1

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    model.fit(X_train_fold, y_train_fold)
    print("fold " + str(fold_no) + " accuracy: " +  str(model.score(X_test_fold, y_test_fold)))
    fold_no += 1
    lst_accu_stratified.append(model.score(X_test_fold, y_test_fold))

print('')
print('Maximum Accuracy',max(lst_accu_stratified))
print('Minimum Accuracy:',min(lst_accu_stratified))
print('Overall Accuracy:', str(round(sum(lst_accu_stratified) / len(lst_accu_stratified), 4) ))

fold 1 accuracy: 0.975
fold 2 accuracy: 0.9375
fold 3 accuracy: 0.9625
fold 4 accuracy: 1.0
fold 5 accuracy: 1.0

Maximum Accuracy 1.0
Minimum Accuracy: 0.9375
Overall Accuracy: 0.975
